In [10]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [24]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_test[1]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [21]:
x_train.shape # (60000, 28, 28)
x_test.shape

(10000, 28, 28)

In [ ]:
"""
MNIST veri kümesindeki rakamların 0'dan 9'a kadar 10 farklı sınıfa ait olduğunu biliyoruz çünkü MNIST veri kümesi,
el yazısı rakamlarının siyah-beyaz görüntülerini içerir ve bu rakamlar 0'dan 9'a kadar olan rakamlardır.

Her bir rakam, bir sınıfı temsil eder. Örneğin, 0 rakamı bir sınıfı, 1 rakamı bir sınıfı, ve böyle devam eder.
Dolayısıyla, tanınacak olan her rakam için bir sınıf bulunmaktadır.

Modelin çıktı katmanında 10 nöron bulunmasının nedeni budur: her bir nöron, bir sınıfı temsil eder ve modelin çıktısını,
verilen görüntünün hangi sınıfa ait olduğunu tahmin etmek için kullanır. Bu nedenle, sınıflandırma problemlerinde genellikle
çıktı katmanında sınıf sayısı kadar nöron bulunur. MNIST veri kümesi için bu sayı 10'dur çünkü 10 farklı rakam sınıfı vardır.
"""
"""
tf.keras.layers.Dense(128, activation='relu'): İkinci katman, 128 nörona sahip bir yoğun (dense) katmandır. "relu" aktivasyon
fonksiyonu bu katmanda kullanılır. ReLU, girişin negatif olduğu durumlarda sıfır çıktı verirken, pozitif değerlerde giriş değerini
doğrudan çıktı olarak verir.
"""

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), #resimler 28*28 o yuzden
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [13]:
"""
x_train[:1]: x_train veri kümesinin ilk örneğini seçer. Bu, genellikle bir görüntüyü temsil eder. Modelin tahmin yapacağı veri bu görüntüdür.
model(x_train[:1]): Model, seçilen bu örneği giriş olarak alır ve tahmin yapar. model burada oluşturulan sinir ağı modelini temsil eder.
Bu, TensorFlow tarafından tanımlanan bir modeldir. Model, giriş verisini alır, katmanlardan geçirir ve sonuçta bir çıktı üretir.

.numpy(): Modelin çıktılarını bir NumPy dizisine dönüştürür. Bu, çıktıları daha kolay işlemek ve analiz etmek için yapılır.
"""
# Her örnek için model, her sınıf için bir tane olmak üzere bir logit vektörü veya log- ord skorları döndürür.

predictions = model(x_train[:1]).numpy()
predictions

array([[-0.2766955 ,  0.04812229,  0.24878395,  0.7426739 ,  0.05020103,
        -0.08325442,  0.17418106,  0.15569311,  0.40246803, -0.27405155]],
      dtype=float32)

In [14]:
# tf.nn.softmax işlevi, bu logitleri her sınıf için olasılıklara dönüştürür:
tf.nn.softmax(predictions).numpy()

array([[0.06438294, 0.08909172, 0.10888889, 0.17843385, 0.08927711,
        0.07812341, 0.10106108, 0.09920984, 0.12697779, 0.06455338]],
      dtype=float32)

In [15]:
"""
  Bir logit vektörü ve bir True indeksi alan ve her örnek için bir skaler kayıp döndüren losses.SparseCategoricalCrossentropy
  kullanarak eğitim için bir kayıp işlevi tanımlayın.
"""

"""
Bu kayıp, gerçek sınıfın negatif log olasılığına eşittir: Model doğru sınıftan eminse kayıp sıfırdır.
Bu eğitimsiz model rastgeleye yakın olasılıklar verir (her sınıf için 1/10), bu nedenle ilk kayıp -tf.math.log(1/10) ~= 2.3 yakın olmalıdır.
"""

"""
Softmax Aktivasyonu ile Uyumluluk: Softmax aktivasyon fonksiyonu ile birlikte kullanıldığında, SparseCategoricalCrossentropy kayıp
fonksiyonu, modelin çıktılarını olasılık dağılımlarına dönüştürür. Bu, modelin doğru sınıflandırma kararları almasını kolaylaştırır.
"""

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [18]:
loss_fn(y_train[:1], predictions).numpy()

2.5494657

In [ ]:
"""Model.compile, TensorFlow'da bir modeli derlemek için kullanılan bir yöntemdir. Bu yöntem, modelin optimize ediciyi (optimizer),
kayıp fonksiyonunu (loss function) ve metrikleri (metrics) belirtmenizi sağlar. İşte model.compile ile belirtilen parametrelerin
açıklamaları:

optimizer='adam': Bu parametre, modelin optimize ediciyi belirler. Optimize edici, modelin eğitim sırasında parametrelerini
güncellemek için kullanılan algoritmayı ifade eder. 'adam', adaptif moment tahminini (Adaptive Moment Estimation) kullanan yaygın
bir optimize edici türüdür. Adam, genellikle derin öğrenme modellerinde iyi performans gösterir.

loss=loss_fn: Bu parametre, modelin kayıp fonksiyonunu belirler. Kayıp fonksiyonu, modelin tahminlerinin gerçek etiketlere ne kadar
uygun olduğunu ölçer. Daha önce belirlediğimiz loss_fn fonksiyonu, SparseCategoricalCrossentropy kayıp fonksiyonudur ve sınıflandırma
problemleri için uygun bir seçenektir.

metrics=['accuracy']: Bu parametre, modelin değerlendirilecek metrikleri belirler. Metrikler, modelin performansını ölçmek için
kullanılan ölçütlerdir. 'accuracy', modelin doğruluk oranını ifade eder, yani doğru olarak sınıflandırılan örneklerin oranını gösterir.
Modelin eğitimi sırasında doğruluk metriğini izlemek, modelin performansını değerlendirmenin önemli bir yoludur."""

In [19]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'] )


In [22]:
"""
x_train: Eğitim veri seti özelliklerini (giriş verilerini) temsil eder.
y_train: Eğitim veri seti etiketlerini (hedef çıktıları) temsil eder.
epochs: Modelin eğitim veri setini kaç kez geçeceğini belirler.

epochs, bir derin öğrenme modelinin eğitim sırasında tam bir eğitim veri setini kaç kez geçeceğini belirten bir parametredir.
"""
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 2ms/step - loss: 0.2946 - accuracy: 0.9144
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1415 - accuracy: 0.9578
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1052 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0854 - accuracy: 0.9737
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0714 - accuracy: 0.9774


In [23]:
"""
x_test: Değerlendirilecek test veri seti özelliklerini (giriş verilerini) temsil eder.
y_test: Değerlendirilecek test veri seti etiketlerini (hedef çıktıları) temsil eder.
verbose: Değerlendirme sırasında çıktıların ne kadar detaylı olacağını belirler. 0, sessiz modu; 1, ilerleme çubuğunu gösterir; 2,
her epoch sonunda bir satır çıktı verir.
"""
model.evaluate(x_test, y_test, verbose=2)


313/313 - 1s - loss: 0.0752 - accuracy: 0.9766 - 648ms/epoch - 2ms/step


[0.0752449557185173, 0.9765999913215637]

In [27]:
"""
tf.keras.Sequential: Bir dizi katmanı birbirine bağlamak için kullanılan bir Keras modeli sınıfıdır. Burada,
yeni bir model oluşturulurken bu sınıf kullanılıyor.

model: Daha önce eğitilmiş olan model adlı model, bu yeni oluşturulan modelin ilk katmanı olarak ekleniyor.
tf.keras.layers.Softmax(): Softmax aktivasyon fonksiyonunu uygulayan bir katman oluşturuluyor. Bu katman, modelin
çıktılarını olasılık dağılımlarına dönüştürmek için kullanılır. Softmax fonksiyonu, her bir sınıf için bir olasılık değeri üretir,
böylece toplam olasılıkların toplamı 1'e eşit olur.
"""
probability_model = tf.keras.models.Sequential([
    model,
    tf.keras.layers.Softmax()
])


In [30]:
probability_model(x_test[:5])


array([[1.00846933e-07, 1.11819949e-08, 3.37211532e-05, 1.42521589e-04,
        2.27120447e-10, 9.43186365e-08, 4.19243199e-13, 9.99819815e-01,
        2.38370390e-07, 3.49590232e-06],
       [1.55088475e-08, 4.17104638e-05, 9.99901056e-01, 4.92848376e-05,
        9.60399969e-17, 2.18075206e-06, 1.34368435e-07, 6.70439073e-13,
        5.76191178e-06, 7.24976262e-15],
       [7.94517348e-07, 9.97713089e-01, 1.04731465e-04, 7.67417951e-05,
        1.31100009e-04, 3.67659231e-05, 2.58619657e-05, 1.35312532e-03,
        5.47991716e-04, 9.66210337e-06],
       [9.99654889e-01, 1.47740495e-08, 8.57925988e-05, 2.68685585e-06,
        2.13296880e-06, 4.20142851e-06, 2.26779608e-04, 5.72382532e-06,
        1.03391167e-05, 7.54658458e-06],
       [8.82279323e-07, 1.49896522e-08, 4.18197169e-06, 1.12909888e-07,
        9.91998136e-01, 8.90854679e-08, 1.55861653e-06, 4.29336287e-05,
        1.64352954e-07, 7.95190502e-03]], dtype=float32)